# Process relaxed crystal results

This Notebook serves as a step-by-step manager for processing the relax and crystal space group calculations to generate relaxed_crystal records.

Note that iprPy.analysis.process_all_relaxations() does the exact same thing as this Notebook.  

__Library imports__

In [2]:
import sys
from pathlib import Path

import numpy as np

import pandas as pd

import iprPy

In [3]:
from iprPy.analysis import match_reference_prototype, get_isolated_atom_energies, process_relaxations

## Specify input terms

In [4]:
csv_root_dir = Path('E:/website/IPR-website/potentials/entry')
database = iprPy.load_database('master')
verbose = False

## Load all input records used to process

In [4]:
all_ref_df = database.get_records_df(style='reference_crystal')
print(len(all_ref_df), 'reference_crystal records found')

6587 reference_crystal records found


In [5]:
all_evsr_records, all_evsr_df = database.get_records(style='calculation_E_vs_r_scan', return_df=True)
print(len(all_evsr_df), 'E_vs_r_scan results found')

28466 E_vs_r_scan results found


In [6]:
all_box_df = database.get_records_df(style='calculation_relax_box')
print(len(all_box_df[all_box_df.branch=='main']), 'relax_box:main results found')

106652 relax_box:main results found


In [7]:
all_static_df = database.get_records_df(style='calculation_relax_static')
print(len(all_static_df[all_static_df.branch=='main']), 'relax_static:main results found')

106652 relax_static:main results found


In [8]:
all_dynamic_df = database.get_records_df(style='calculation_relax_dynamic')
print(len(all_dynamic_df[all_dynamic_df.branch=='main']), 'relax_dynamic:main results found')

106652 relax_dynamic:main results found


In [9]:
print(len(all_dynamic_df[(all_dynamic_df.branch=='main') & (all_dynamic_df.status=='finished')]), 'relax_static:from_dynamic expected')
print(len(all_static_df[all_static_df.branch=='from_dynamic']), 'relax_static:from_dynamic results found')

98126 relax_static:from_dynamic expected
98114 relax_static:from_dynamic results found


In [10]:
all_spg_df = database.get_records_df(style='calculation_crystal_space_group')
print(len(all_spg_df[all_spg_df.branch == 'relax']), 'crystal_space_group:relax results found')
print(len(all_spg_df[all_spg_df.branch == 'prototype']), 'crystal_space_group:prototype results found')
print(len(all_spg_df[all_spg_df.branch == 'reference']), 'crystal_space_group:reference results found')

215775 crystal_space_group:relax results found
19 crystal_space_group:prototype results found
6728 crystal_space_group:reference results found


In [11]:
ref_proto_df = match_reference_prototype(database, all_spg_df=all_spg_df)
print(len(ref_proto_df), 'reference-prototype relations found')

6728 reference-prototype relations found


In [12]:
iso_energy_df = get_isolated_atom_energies(database)
print(len(iso_energy_df), 'isolated atom energies compiled')

1446 isolated atom energies compiled


## Identify all potentials in the results

In [13]:
def combine_pot_info(series):
    return f'{series.potential_id}:{series.potential_key}:{series.potential_LAMMPS_id}:{series.potential_LAMMPS_key}'

pots = np.unique(
    all_evsr_df.apply(combine_pot_info, axis=1).tolist() +
    all_box_df.apply(combine_pot_info, axis=1).tolist() +
    all_static_df.apply(combine_pot_info, axis=1).tolist() +
    all_dynamic_df.apply(combine_pot_info, axis=1).tolist() 
)
pot_info = []
for pot in pots:
    terms = pot.split(':')
    pinfo = {}
    pinfo['potential_id'] = terms[0]
    pinfo['potential_key'] = terms[1]
    pinfo['potential_LAMMPS_id'] = terms[2]
    pinfo['potential_LAMMPS_key'] = terms[3]
    pot_info.append(pinfo)
pot_info = pd.DataFrame(pot_info)
pot_info
print(len(pot_info), 'potentials to check')

1013 potentials to check


## Iterate over all potentials

In [16]:
verbose=False
for i in pot_info.index:
    pot = pot_info.loc[i]
    print(pot.potential_id)
    print(pot.potential_LAMMPS_id)

    potential_LAMMPS_key = pot.potential_LAMMPS_key
    potential_key = pot.potential_key
    try:
        potential = database.potdb.get_lammps_potential(key=potential_LAMMPS_key, potkey=potential_key)
        process_relaxations(database, potential,
                            ref_proto_df = ref_proto_df,
                            iso_energy_df = iso_energy_df,
                            all_ref_df = all_ref_df,
                            all_evsr_records = all_evsr_records,
                            all_evsr_df = all_evsr_df,
                            all_box_df = all_box_df,
                            all_static_df = all_static_df,
                            all_dynamic_df = all_dynamic_df,
                            all_spg_df = all_spg_df,
                            csv_root_dir = csv_root_dir,
                            verbose = verbose)
    except ValueError as e:
        print('!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!')
        print(e)

    print()
    if verbose:
        print()
        print()

1958--Bernardes-N--Ar
LJ_Shifted_Bernardes_1958HighCutoff_Ar__MO_242741380554_003

1958--Bernardes-N--Ar
LJ_Shifted_Bernardes_1958HighCutoff_Ar__MO_242741380554_004

1958--Bernardes-N--Ar
LJ_Shifted_Bernardes_1958LowCutoff_Ar__MO_720819638419_003

1958--Bernardes-N--Ar
LJ_Shifted_Bernardes_1958LowCutoff_Ar__MO_720819638419_004

1958--Bernardes-N--Ar
LJ_Shifted_Bernardes_1958MedCutoff_Ar__MO_126566794224_003

1958--Bernardes-N--Ar
LJ_Shifted_Bernardes_1958MedCutoff_Ar__MO_126566794224_004

1958--Bernardes-N--Ar
LJ_Smoothed_Bernardes_1958_Ar__MO_764178710049_001

1958--Bernardes-N--Kr
LJ_Shifted_Bernardes_1958HighCutoff_Kr__MO_923895531627_003

1958--Bernardes-N--Kr
LJ_Shifted_Bernardes_1958HighCutoff_Kr__MO_923895531627_004

1958--Bernardes-N--Kr
LJ_Shifted_Bernardes_1958LowCutoff_Kr__MO_995724792024_003

1958--Bernardes-N--Kr
LJ_Shifted_Bernardes_1958LowCutoff_Kr__MO_995724792024_004

1958--Bernardes-N--Kr
LJ_Shifted_Bernardes_1958MedCutoff_Kr__MO_984281096460_003

1958--Bernardes-N--K


1959--Girifalco-L-A-Weizer-V-G--Mo
Morse_Shifted_GirifalcoWeizer_1959MedCutoff_Mo__MO_534363225491_004

1959--Girifalco-L-A-Weizer-V-G--Na
Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Na__MO_587469264453_003

1959--Girifalco-L-A-Weizer-V-G--Na
Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Na__MO_587469264453_004

1959--Girifalco-L-A-Weizer-V-G--Na
Morse_Shifted_GirifalcoWeizer_1959LowCutoff_Na__MO_707981543254_003

1959--Girifalco-L-A-Weizer-V-G--Na
Morse_Shifted_GirifalcoWeizer_1959LowCutoff_Na__MO_707981543254_004

1959--Girifalco-L-A-Weizer-V-G--Na
Morse_Shifted_GirifalcoWeizer_1959MedCutoff_Na__MO_636041334617_003

1959--Girifalco-L-A-Weizer-V-G--Na
Morse_Shifted_GirifalcoWeizer_1959MedCutoff_Na__MO_636041334617_004

1959--Girifalco-L-A-Weizer-V-G--Ni
Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Ni__MO_381861218831_003

1959--Girifalco-L-A-Weizer-V-G--Ni
Morse_Shifted_GirifalcoWeizer_1959HighCutoff_Ni__MO_381861218831_004

1959--Girifalco-L-A-Weizer-V-G--Ni
Morse_Shifted_GirifalcoW


1988--Khor-K-E-Das-Sarma-S--C
ThreeBodyBondOrder_KDS_KhorDasSarma_1988_C__MO_454320668790_000
!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
Some relax_static calculations not finished!

1988--Khor-K-E-Das-Sarma-S--Ge
ThreeBodyBondOrder_KDS_KhorDasSarma_1988_Ge__MO_216597146527_000
!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
Some relax_static calculations not finished!

1988--Khor-K-E-Das-Sarma-S--Si
ThreeBodyBondOrder_KDS_KhorDasSarma_1988_Si__MO_722489435928_000
!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
Some relax_static calculations not finished!

1988--Tersoff-J--C
Tersoff_LAMMPS_Tersoff_1988_C__MO_579868029681_002

1988--Tersoff-J--Si-b
1988--Tersoff-J--Si-b--LAMMPS--ipr1

1988--Tersoff-J--Si-b
Tersoff_LAMMPS_Tersoff_1988T2_Si__MO_245095684871_002

1988--Tersoff-J--Si-c
1988--Tersoff-J--Si-c--LAMMPS--ipr1

1988--Tersoff-J--Si-c
Tersoff_LAMMPS_Tersoff_1988T3_Si__MO_186459956893_002

1989--Adams-J-B-Foiles-S-M-Wolfer-W-G--Ag-Au-Cu-Ni-Pd-Pt
1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS-


1999--Liu-X-Y-Liu-C-L-Borucki-L-J--Al-Cu
EAM_Dynamo_LiuLiuBorucki_1999_AlCu__MO_020851069572_000

1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al
1999--Mishin-Y--Al--LAMMPS--ipr1

1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Al
EAM_Dynamo_MishinFarkasMehl_1999_Al__MO_651801486679_005

1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Ni
1999--Mishin-Y--Ni--LAMMPS--ipr1

1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Ni
EAM_Dynamo_MishinFarkasMehl_1999_Ni__MO_400591584784_005

2000--Landa-A-Wynblatt-P-Siegel-D-J-et-al--Al-Pb
2000--Landa-A--Al-Pb--LAMMPS--ipr1

2000--Landa-A-Wynblatt-P-Siegel-D-J-et-al--Al-Pb
EAM_Dynamo_LandaWynblattSiegel_2000_AlPb__MO_699137396381_005

2000--Lenosky-T-J-Sadigh-B-Alonso-E-et-al--Si
2000--Lenosky-T-J--Si--LAMMPS--ipr1

2000--Sturgeon-J-B-Laird-B-B--Al
2000--Sturgeon-J-B--Al--LAMMPS--ipr1

2000--Sturgeon-J-B-Laird-B-B--Al
EAM_Dynamo_SturgeonLaird_2000_Al__MO_120808805541_005

2001--Lee-B-J-Baskes-M-I-Kim-H-Cho


2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Cu
2004--Zhou-X-W--Cu--LAMMPS--ipr2

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Cu
EAM_Dynamo_ZhouJohnsonWadley_2004NISTretabulation_Cu__MO_759493141826_000

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Cu
EAM_Dynamo_ZhouJohnsonWadley_2004_Cu__MO_127245782811_005

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Fe
2004--Zhou-X-W--Fe--LAMMPS--ipr1

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Fe
2004--Zhou-X-W--Fe--LAMMPS--ipr2

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Fe
EAM_Dynamo_ZhouJohnsonWadley_2004NISTretabulation_Fe__MO_681088298208_000

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Fe
EAM_Dynamo_ZhouJohnsonWadley_2004_Fe__MO_650279905230_005

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Mg
2004--Zhou-X-W--Mg--LAMMPS--ipr1

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Mg
2004--Zhou-X-W--Mg--LAMMPS--ipr2

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Mg
EAM_Dynamo_ZhouJohnsonWadley_2004NISTretabulation_Mg__MO_894868634445_000

2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Mg
EA

...


2017--Purja-Pun-G-P-Mishin-Y--Si
ThreeBodyBondOrder_PPM_PurjaPunMishin_2017_Si__MO_566683736730_000

2017--Smirnova-D-E-Starikov-S-V--Zr-Nb
2017--Smirnova-D-E--Zr-Nb--LAMMPS--ipr1

2017--Tehranchi-A-Curtin-W-A--Ni-H
EAM_Dynamo_TehranchiCurtin_2010_NiH__MO_535504325462_003

2017--Wang-P-Xu-S-Liu-J-et-al--TWIP
2017--Wang-P--TWIP--LAMMPS--ipr1

2017--Wang-P-Xu-S-Liu-J-et-al--TWIP
2017--Wang-P--TWIP--LAMMPS--ipr2

2017--Wen-M-Shirodkar-S-N-Plechac-P-et-al--Mo-S
SW_MX2_WenShirodkarPlechac_2017_MoS__MO_201919462778_001

2017--Wu-C-Lee-B-J-Su-X--Ni-Cr-Fe
2017--Wu-C--Ni-Cr-Fe--LAMMPS--ipr1

2017--Wu-C-Lee-B-J-Su-X--Ni-Cr
2017--Wu-C--Ni-Cr--LAMMPS--ipr1

2017--Wu-C-Lee-B-J-Su-X--Ni-Fe
2017--Wu-C--Ni-Fe--LAMMPS--ipr1

2017--Zhou-X-W-Jones-R-E-Chu-K--In-Ga-N
2017--Zhou-X-W--In-Ga-N--LAMMPS--ipr1

2018--Byggmastar-J-Hodille-E-A-Ferro-Y-Nordlund-K--Be-O
2018--Byggmastar-J--Be-O--LAMMPS--ipr1

2018--Choi-W-M-Jo-Y-H-Sohn-S-S-et-al--Co-Ni-Cr-Fe-Mn
2018--Choi-W-M--Co-Ni-Cr-Fe-Mn--LAMMPS--ipr1

2018--D

!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
Some relax_static:from_dynamic calculations not finished!

2020--Byggmastar-J-Granberg-F--Fe
2020--Byggmastar-J--Fe--LAMMPS--ipr1

2020--Farkas-D-Caro-A--Fe-Ni-Cr-Co-Al
2020--Farkas-D--Fe-Ni-Cr-Co-Al--LAMMPS--ipr1

2020--Groger-R-Vitek-V-Dlouhy-A--Co-Cr-Fe-Mn-Ni
2020--Groger-R--Co-Cr-Fe-Mn-Ni--LAMMPS--ipr1

2020--Jeong-G-U-Lee-B-J--Pd-C
2020--Jeong-G-U--Pd-C--LAMMPS--ipr1

2020--Jeong-G-U-Lee-B-J--Pt-C
2020--Jeong-G-U--Pt-C--LAMMPS--ipr1

2020--Li-X-G-Chen-C-Zheng-H-et-al--Nb-Ta-W-Mo
SNAP_LiChenZheng_2019_NbTaWMo__MO_560387080449_000

2020--Miraz-A-S-M-Dhariwal-N-Meng-W-J-et-al--Cu-N-Ti
2020--Miraz-A-S-M--Cu-N-Ti--LAMMPS--ipr1

2020--Mori-H-Ozaki-T--Fe
2020--Mori-H--Fe--LAMMPS--ipr1

2020--Oh-S-H-Seol-D-Lee-B-J--Co-Ti
2020--Oh-S-H--Co-Ti--LAMMPS--ipr1

2020--Oh-S-H-Seol-D-Lee-B-J--Co-V
2020--Oh-S-H--Co-V--LAMMPS--ipr1

2020--Purja-Pun-G-P-Yamakov-V-Hickman-J-et-al--Al
2020--Purja-Pun-G-P--Al--LAMMPS--ipr1

2020--Starikov-S-Gordeev-I-Lysogorsk

Extra (duplicate?) calculations found:
['0e618b70-1f79-4955-a814-df81a81f5e04', 'd00a2042-ef73-421b-aaeb-3904b7e1b429']
Extra (duplicate?) calculations found:
['384f433b-4509-4b2d-93af-584caa3df736', 'b0d32826-4588-4a6c-adb7-e398b727c722']
Extra (duplicate?) calculations found:
['1ebd6746-d97d-47dc-a9c6-5ad185ce9dd9', 'd28ddd6f-c10a-43aa-925f-dec6e4e1f9c3']
Extra (duplicate?) calculations found:
['8e980cc4-d12f-4681-bf58-10bf37a1409f', 'e78c4030-ec7f-41d7-884e-ab9abbb69000']
Extra (duplicate?) calculations found:
['0d1f76f3-750a-44d1-9e25-dea2fba2fc62', 'd5b132b4-7f37-4413-bea9-0fcd94adb1ff']
Extra (duplicate?) calculations found:
['02ad8beb-6671-4e86-8ff0-1a62d7808acc', 'd51517ab-110b-4d7c-8819-d01248210724']
Extra (duplicate?) calculations found:
['27906fe2-fcf3-4431-844f-c5b4b9c36012', '8fc9e1b9-7bb7-4e19-b9dc-32cbc7026ba9']
Extra (duplicate?) calculations found:
['13530bf7-0395-49ae-9ceb-46c2f931c017', '28f21d83-b2d7-48c8-b4a9-6632e2fff9d8']
Extra (duplicate?) calculations found:
[


2021--Starikov-S-Smirnova-D-Pradhan-T-et-al--Fe
2021--Starikov-S--Fe--LAMMPS--ipr2
Extra (duplicate?) calculations found:
['6ed7b594-eea0-4e94-8193-77d6616987e5', '8172143f-0023-4647-9e32-c48c97c48dff']
Extra (duplicate?) calculations found:
['ae7cdb3c-ad51-4524-8f1e-d6e254a5dd03', 'b31012f0-579c-44a4-aa86-e49837e762f1']
Extra (duplicate?) calculations found:
['579f7c7b-b9d8-40b6-9cc8-65a895335657', '7c9228f6-03f8-4253-b085-7858f85812a3']
Extra (duplicate?) calculations found:
['25f65983-a2be-41d8-8e84-bbb4f0717b40', 'bbbe81d4-ed8a-46cd-91a3-d3ea9c73492e']
Extra (duplicate?) calculations found:
['0427d7b6-381b-4fd6-91bb-f02f7f601325', '4efe01d6-8fb8-4a07-9328-fe2e4779cc11']
Extra (duplicate?) calculations found:
['53b8d701-65d6-49c0-9ee0-173b368c45c6', 'eccee0c7-0393-4581-94fa-d1354dd5f58d']
Extra (duplicate?) calculations found:
['1ec39e92-71f6-49e4-85da-56f82137b798', '895ad916-fef7-4dad-ab6f-42df71be24dd']
Extra (duplicate?) calculations found:
['1a508950-fa98-4937-b112-48894f5b144


2021--Wang-G-Xu-Y-Qian-P-Su-Y--Au-Rh
2021--Wang-G--Au-Rh--LAMMPS--ipr1

2021--Wen-M--Fe-H
2021--Wen-M--Fe-H--LAMMPS--ipr1

2021--Zhou-H-Dickel-D-E-Baskes-M-I-et-al--Bi
2021--Zhou-H--Bi--LAMMPS--ipr1
!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
No successful crystal relaxations found!

2021--Zhou-X-W-Bartelt-N-C-Sills-R-B--Pd-H-He
2021--Zhou-X-W--Pd-H-He--LAMMPS--ipr1
!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
No successful crystal relaxations found!

2022--Alvi-S-M-A-A-Faiyad-A-Munshi-M-A-M-et-al--Ag-Au
2022--Alvi-S-M-A-A--Ag-Au--LAMMPS--ipr1

2022--Alvi-S-M-A-A-Faiyad-A-Munshi-M-A-M-et-al--Ag
2022--Alvi-S-M-A-A--Ag--LAMMPS--ipr1

2022--Alvi-S-M-A-A-Faiyad-A-Munshi-M-A-M-et-al--Au
2022--Alvi-S-M-A-A--Au--LAMMPS--ipr1

2022--Hiremath-P-Melin-S-Bitzek-E-Olsson-P-A-T--W
2022--Hiremath-P--W--LAMMPS--ipr1
!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!
Some crystal_space_group calculations not finished!

2022--Lin-Y-S-Purja-Pun-G-P-Mishin-Y--Ta
2022--Lin-Y-S--Ta--LAMMPS--ipr1

2022--Mahata-A-M

## Iterate over specified potentials

This section iterates over only specified potentials

In [5]:
potential_LAMMPS_ids = [
    '2019--Yang-C--Nb--LAMMPS--ipr1'
]

verbose=False
for potential_LAMMPS_id in potential_LAMMPS_ids:
    print(potential_LAMMPS_id)

    try:
        potential = database.potdb.get_lammps_potential(id=potential_LAMMPS_id)
        process_relaxations(database, potential,
                            #ref_proto_df = ref_proto_df,
                            #iso_energy_df = iso_energy_df,
                            #all_ref_df = all_ref_df,
                            #all_evsr_records = all_evsr_records,
                            #all_evsr_df = all_evsr_df,
                            #all_box_df = all_box_df,
                            #all_static_df = all_static_df,
                            #all_dynamic_df = all_dynamic_df,
                            #all_spg_df = all_spg_df,
                            csv_root_dir = csv_root_dir,
                            verbose = verbose)
    except ValueError as e:
        print('!!!!!!!!!!!!!!!   FAILED   !!!!!!!!!!!!!!!!')
        print(e)

    print()
    if verbose:
        print()
        print()

2019--Yang-C--Nb--LAMMPS--ipr1



## Current summary/issues

### No successful crystal relaxations found!

- SW_StillingerWeber_1985_Si__MO_405512056662_006
- SW_WangStroudMarkworth_1989_CdTe__MO_786496821446_001
- 1990--Ackland-G-J--Cu--LAMMPS--ipr1
- SW_BalamaneHaliciogluTiller_1992_Si__MO_113686039439_005
- SW_BalamaneHauchShi_2017Brittle_Si__MO_381114941873_003
- SW_HauchHollandMarder_1999Brittle_Si__MO_119167353542_005
- SW_BereSerra_2006_GaN__MO_861114678890_001
- TT_Modified_HellmannBichVogel_2007_He__MO_126942667206_002
- 2009--Kim-H-K--Fe-Ti-C--LAMMPS--ipr1
- 2009--Zhakhovskii-V-V--Al--LAMMPS--ipr1
- 2009--Zhakhovskii-V-V--Au--LAMMPS--ipr1
- 2012--Jelinek-B--Al-Si-Mg-Cu-Fe--LAMMPS--ipr1
- SW_LeeHwang_2012GGA_Si__MO_040570764911_001
- SW_LeeHwang_2012LDA_Si__MO_517338295712_001
- 2013--Gao-H--AgTaO3--LAMMPS--ipr1
- SW_ZhouWardMartin_2013_CdTeZnSeHgS__MO_503261197030_003
- 2014--Liyanage-L-S-I--Fe-C--LAMMPS--ipr1
- SW_ZhangXieHu_2014OptimizedSW1_Si__MO_800412945727_005
- SW_ZhangXieHu_2014OptimizedSW2_Si__MO_475612090600_005
- 2015--Broqvist-P--Ce-O--LAMMPS--ipr1
- LJ_ElliottAkerson_2015_Universal__MO_959249795837_003
- 2015--Ko-W-S--Ni-Ti--LAMMPS--ipr1
- 2015--Pascuet-M-I--Al-U--LAMMPS--ipr1
- 2021--Zhou-H--Bi--LAMMPS--ipr1
- 2021--Zhou-X-W--Pd-H-He--LAMMPS--ipr1

### Some relax_static calculations not finished!

- ThreeBodyBondOrder_KDS_KhorDasSarma_1988_C__MO_454320668790_000
- ThreeBodyBondOrder_KDS_KhorDasSarma_1988_Ge__MO_216597146527_000
- ThreeBodyBondOrder_KDS_KhorDasSarma_1988_Si__MO_722489435928_000
- EAM_Mendelev_2019_CuZr__MO_945018740343_000
- DUNN_WenTadmor_2019v1_C__MO_584345505904_000
- DUNN_WenTadmor_2019v2_C__MO_956135237832_000
- DUNN_WenTadmor_2019v3_C__MO_714772088128_000


### Some relax_static:from_dynamic calculations not finished!

- SNAP_LiHuChen_2018_Cu__MO_529419924683_000
- SNAP_LiHuChen_2018_NiMo__MO_468686727341_000
- SNAP_LiChenZheng_2019_NbTaWMo__MO_560387080449_000
- hNN_WenTadmor_2019Grx_C__MO_421038499185_001
- SNAP_ZuoChenLi_2019_Ge__MO_183216355174_000
- SNAP_ZuoChenLi_2019quadratic_Li__MO_041269750353_000
- SNAP_ZuoChenLi_2019quadratic_Ni__MO_263593395744_000
- SNAP_ZuoChenLi_2019_Si__MO_869330304805_000

### Some crystal_space_group calculations not finished!

- 2019--Plummer-G--Ti-Si-C--LAMMPS--ipr1
- 2022--Hiremath-P--W--LAMMPS--ipr1

### No matching LAMMPS potentials found

- EAM_Dynamo_Mendelev_2003_Fe__MO_546673549085_000
- EAM_Dynamo_Mendelev_2007_Zr__MO_848899341753_000

### Missing calculation

- 2011--Daw-M-S--Hf-B--LAMMPS--ipr1
- 2011--Daw-M-S--Zr-B--LAMMPS--ipr1
- LJ_ElliottAkerson_2015_Universal__MO_959249795837_003 
- hNN_WT_WenTadmor_2019Grx_C__MO_421038499185_000 (old pot?)
- SNAP_ZuoChenLi_2019quadratic_Si__MO_721469752060_000


## Merge databases

In [23]:
remote = iprPy.load_database('potentials')
master = iprPy.load_database('master')
iprhub = database

In [25]:
bad = database.get_records('relaxed_crystal', key=key)
len(bad)

34

In [30]:
iprhub.merge_records(master, 'relaxed_crystal', includetar=False, overwrite=False)

166907 records in source
162693 records in destination
4214 records missing from destination
4214 records to try to copy


copying records: 100%|#############################################################| 4214/4214 [06:30<00:00, 10.78it/s]


4214 records added/updated
